In [ ]:
import torch
from torch.utils.cpp_extension import load
import numpy as np
import torch.nn.functional as F
import scipy.sparse as sp
from scipy.spatial import KDTree

differentiable_rapid_raditor = load(
    name="differentiable_rapid_raditor",
    sources=["utils/differentiable_rapid_raditor_kernel.cpp", "utils/differentiable_rapid_raditor_kernel_v4_fine.cu"],
    verbose=True,
)

In [ ]:
# Define the custom Autograd Function
class SimulateFunction(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, sensor_location, source_location, source_p0, source_dx, dt, num_sensors, num_sources, num_times):
        # Call the C++ forward function
        simulate_record = differentiable_rapid_raditor.simulate(
            sensor_location, source_location, source_p0, source_dx, dt, num_sensors, num_sources, num_times)
        
        # Save inputs for backward
        ctx.save_for_backward(sensor_location, source_location, source_p0, source_dx)
        ctx.dt = dt
        ctx.num_sensors = num_sensors
        ctx.num_sources = num_sources
        ctx.num_times = num_times
        
        return simulate_record  # simulate_record：torch.Size([num_sensors * num_times])
    
    @staticmethod
    def backward(ctx, dL_dsimulate_record):
        # dL_dsimulate_record：torch.Size([num_sensors * num_times])
        sensor_location, source_location, source_p0, source_dx = ctx.saved_tensors
        dt = ctx.dt
        num_sensors = ctx.num_sensors
        num_sources = ctx.num_sources
        num_times = ctx.num_times


        # Call the C++ backward function
        grad_source_location, grad_source_p0, grad_source_dx = differentiable_rapid_raditor.simulate_backward(
            sensor_location, source_location, source_p0, source_dx, dL_dsimulate_record.contiguous(), dt, num_sensors, num_sources, num_times
        )
        return None, grad_source_location, grad_source_p0, grad_source_dx, None, None, None, None

# Utility function to use the custom autograd function
def simulate(sensor_location, source_location, source_p0, source_dx, dt, num_sensors, num_sources, num_times):
    return SimulateFunction.apply(sensor_location, source_location, source_p0, source_dx, dt, num_sensors, num_sources, num_times)

In [ ]:
import torch
import numpy as np
from torch.autograd import Function
# from utils.dataset_loader_full_angle import calculate_detector_location, read_one_dat_into_a_matrix
from utils.loss_utils import l2_loss, ssim
import torch.optim as optim
import random
from utils.sliding_ball_model_for_e6_cuda0 import SlidingBallModel  
from tqdm import tqdm
import matplotlib.pyplot as plt
import time
import os

device = torch.device('cuda:0')

def load_point_cloud(file_path):
    balls = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        start = False
        for line in lines:
            if start:
                data = line.split()
                if len(data) == 5:
                    xyz = torch.tensor([float(data[0]), float(data[1]), float(data[2])], requires_grad=True, dtype=torch.float32, device=device)
                    pressure_0 = torch.tensor(float(data[3]), requires_grad=True, dtype=torch.float32, device=device)
                    radius = torch.tensor(np.log(np.exp(float(data[4]))-1), requires_grad=True, dtype=torch.float32, device=device)
                    ball = SlidingBallModel(xyz, pressure_0, radius)
                    balls.append(ball)
            if 'end_header' in line:
                start = True
    return balls

file_path = 'iteration_point_cloud_fine_recon/ball_120_after.ply'
balls = load_point_cloud(file_path)
print(f"{len(balls)} balls loaded from point cloud file.")

# 随机初始化球分布
def generate_random_balls(num_balls, boundaries, res):
    balls = []
    x_min, x_max, y_min, y_max, z_min, z_max = boundaries
    for _ in range(num_balls):
        xyz = torch.tensor([random.uniform(x_min, x_max), random.uniform(y_min, y_max), random.uniform(z_min, z_max)], requires_grad=False, dtype=torch.float32, device=device)
        pressure_0 = torch.tensor(random.uniform(20, 100), requires_grad=True, dtype=torch.float32, device=device)
        radius = torch.tensor(random.uniform(1*res, 6*res), requires_grad=True, dtype=torch.float32, device=device)
        ball = SlidingBallModel(xyz, pressure_0, radius)
        balls.append(ball)
    return balls

# 迭代函数
def run_iterations(balls, pressure_threshold, radius_max_threshold, radius_min_threshold, boundaries):
    new_balls = []
    for ball in balls:
        new_ball = ball.adaptive_density_optimization(pressure_threshold, radius_max_threshold, radius_min_threshold, boundaries)
        if new_ball is not None:
            new_balls.append(new_ball)
    balls = [ball for ball in balls if not ball._is_destroyed] + new_balls
    return balls

# 保存点云数据函数
def save_point_cloud(balls, filename):
    points = []
    for ball in balls:
        if not ball._is_destroyed:
            xyz = ball._xyz.detach().cpu().numpy()  # 转为 numpy 数组
            pressure_0 = ball._pressure_0.item()  # 提取标量值
            radius = ball._radius.item()  # 提取标量值
            points.append([xyz[0], xyz[1], xyz[2], pressure_0, radius])

    with open(filename, "w") as ply_file:
        # 写 PLY 文件头
        ply_file.write("ply\n")
        ply_file.write("format ascii 1.0\n")
        ply_file.write(f"element vertex {len(points)}\n")
        ply_file.write("property float x\n")
        ply_file.write("property float y\n")
        ply_file.write("property float z\n")
        ply_file.write("property float pressure_0\n")
        ply_file.write("property float radius\n")
        ply_file.write("end_header\n")
        
        # 写入每个点的信息
        for point in points:
            ply_file.write(f"{point[0]} {point[1]} {point[2]} {point[3]} {point[4]}\n")

# 参数设置
num_times = 4096
Nt = num_times
res = 0.10e-3
Vs = 1500.0
dt = 25e-9  # [s]
folder_name = "iteration_point_cloud_fine_recon_softplus"
os.makedirs(folder_name, exist_ok=True)
boundaries = (-6.4e-3, 6.4e-3, -6.4e-3, 6.4e-3, -6.4e-3, 6.4e-3)

# 进行初始化，并提取初始化声源的信息
source_num = len(balls)

pressure_threshold = 15.0
radius_max_threshold = np.log(np.exp(3 * res)-1)
radius_min_threshold = np.log(np.exp(0.5 * res)-1)


In [ ]:
# 读取sensor_data_matrix.txt文件
sensor_data_matrix = np.loadtxt('data/simulation_frame_01.txt', delimiter='\t')
real_signal = sensor_data_matrix
real_signal = real_signal[::4, :]
sensor_location = np.loadtxt('data/sensor_location_Sphere2048_Fibonacci.txt', delimiter='\t')
sensor_location = sensor_location[::4, :]
sensor_location = sensor_location*1e-3
sensor_num = sensor_location.shape[0]
print(real_signal.shape)
print(sensor_location.shape)

In [ ]:


# 将数据转为PyTorch张量，并移动到GPU
sensor_location = torch.tensor(sensor_location,dtype=torch.float, device=device)
real_signal = torch.tensor(real_signal,dtype=torch.float)
real_signal_flat = real_signal.flatten()
real_signal_flat = real_signal_flat.to(device)

# 将所有参数按照不同学习率进行分组
def get_optim_params(balls):
    params_xyz = [ball._xyz for ball in balls]
    params_pressure = [ball._pressure_0 for ball in balls]
    params_radius = [ball._radius for ball in balls]
    return [
        {'params': params_xyz, 'lr': 0.000003},
        {'params': params_pressure, 'lr': 0.003},
        {'params': params_radius, 'lr': 0.03},
    ]

optimizer = optim.Adam(get_optim_params(balls), betas=(0.9, 0.999))

# 设置学习率调度器
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

num_iterations = 1000


def build_laplacian_matrix(points, k_neighbors=10):
    """
    points: torch.Tensor, shape [N, 3] (所有球的xyz坐标)
    k_neighbors: 每个点的邻域数
    """
    N = points.shape[0]
    points_np = points.detach().cpu().numpy()  # 转为CPU numpy以加速KNN计算
    
    # 使用KDTree找K近邻
    tree = KDTree(points_np)
    distances, indices = tree.query(points_np, k=k_neighbors + 1)  # 包含自身
    
    # 构建稀疏权重矩阵W (高斯权重)
    rows = np.repeat(np.arange(N), k_neighbors)
    cols = indices[:, 1:].flatten()  # 排除自身
    weights = np.exp(-distances[:, 1:].flatten() ** 2 / (2 * 0.1))  # 0.1为带宽参数
    
    # 构造稀疏矩阵（CSR格式）
    W = sp.csr_matrix((weights, (rows, cols)), shape=(N, N))
    D = sp.diags(W.sum(axis=1).A1)
    L = D - W  # 拉普拉斯矩阵
    
    # 转换为COO格式以获取row和col属性
    L_coo = L.tocoo()
    
    # 转为PyTorch稀疏张量并移到GPU
    indices = torch.tensor(np.vstack([L_coo.row, L_coo.col]), dtype=torch.long, device=device)
    values = torch.tensor(L_coo.data, dtype=torch.float32, device=device)
    L_sparse = torch.sparse_coo_tensor(indices, values, size=L_coo.shape, device=device)
    
    return L_sparse
# 在训练循环前定义拉普拉斯参数
lambda_reg = 0.5  # 正则化强度
k_neighbors = 16     # 邻域数


# 开始迭代训练
for iter in range(num_iterations):
    start_time = time.time()  # 记录开始时间
    optimizer.zero_grad()


    source_location = torch.stack([ball._xyz for ball in balls])
    # 构建拉普拉斯矩阵（仅在点云结构变化时更新）
    if iter % 10 == 0 or iter == 0:
        L = build_laplacian_matrix(source_location, k_neighbors)
    
    # 计算拉普拉斯正则项
    laplacian_loss = torch.sum((L @ source_location) ** 2)

    # 使用当前球的参数传递给simulate函数
    source_location = torch.stack([ball._xyz for ball in balls]).flatten()
    source_p0 = torch.stack([ball._pressure_0 for ball in balls]).flatten()
    radius_0 = torch.stack([ball._radius for ball in balls]).flatten()
    radius_0 = F.softplus(radius_0)

    simulate_record = simulate(sensor_location, source_location, source_p0, radius_0, dt, sensor_num, source_num, num_times)  
    data_loss = l2_loss(simulate_record, real_signal_flat)
    
    # 总损失
    total_loss = data_loss + lambda_reg * laplacian_loss
    total_loss.backward()
    optimizer.step()
    lr_scheduler.step()
    
    # print(f"迭代 {iter + 1}，损失: {loss.item()}")
    end_time = time.time()  # 记录结束时间
    iteration_time = end_time - start_time  # 计算时间差
    print(f"迭代 {iter + 1}，损失: {total_loss.item()}，时间: {iteration_time}")
    # 更新每个球的参数，以确保它们在相应的实例中
    for ball in balls:
        ball._xyz.data = ball._xyz.data.requires_grad_(True)
        ball._pressure_0.data = ball._pressure_0.data.requires_grad_(True)
        ball._radius.data = ball._radius.data.requires_grad_(True)

    # 每10次迭代前保存点云数据到文件
    if (iter + 1) % 10 == 0:
        filename = f"ball_{iter + 1}_before.ply"
        full_path = os.path.join(folder_name, filename)
        save_point_cloud(balls, full_path)
        print(f"Point cloud saved to {filename}")

    # 每10次迭代后保存点云数据到文件
    if (iter + 1) % 10 == 0:
        filename = f"ball_{iter + 1}_after.ply"
        full_path = os.path.join(folder_name, filename)
        save_point_cloud(balls, full_path)
        print(f"Point cloud saved to {filename}")